In [6]:
!pip install pytest_notebook

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 10.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.8/366.8 kB 11.8 MB/s eta 0:00:00
  Using cached jupyter_server_mathjax-0.2.6-py3-none-any.whl (3.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.9/426.9 kB 11.3 MB/s eta 0:00:000:00:01
  U

  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Using cached arrow-1.2.3-py3-none-any.whl (66 kB)


In [7]:
import ipytest
import pytest

ipytest.config.magics_flags = ['-qq']
ipytest.config(rewrite_asserts=True, magics=True)


{'rewrite_asserts': True,
 'magics': True,
 'clean': '[Tt]est*',
 'addopts': (),
 'run_in_thread': False,
 'defopts': 'auto',
 'display_columns': 100,
 'raise_on_error': False}

In [12]:
def validate_dunder_operators(obj):
    dunder_operators = [
        "__eq__",
        "__ne__",
        "__lt__",
        "__le__",
        "__gt__",
        "__ge__",
    ]

    missing_operators = [op for op in dunder_operators if not hasattr(obj, op)]

    if missing_operators:
        return False, missing_operators
    else:
        return True, None

class Point:
    def __init__(self, value):
        self.value = value

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.value == other.value
        return NotImplemented

    def __ne__(self, other):
        if isinstance(other, Point):
            return self.value != other.value
        return NotImplemented

    def __lt__(self, other):
        if isinstance(other, Point):
            return self.value < other.value
        return NotImplemented

    def __le__(self, other):
        if isinstance(other, Point):
            return self.value <= other.value
        return NotImplemented

    def __gt__(self, other):
        if isinstance(other, Point):
            return self.value > other.value
        return NotImplemented

    def __ge__(self, other):
        if isinstance(other, Point):
            return self.value >= other.value
        return NotImplemented

    def __add__(self, other):
        if isinstance(other, Point):
            return Point(self.value + other.value)
        return NotImplemented

    def __sub__(self, other):
        if isinstance(other, Point):
            return Point(self.value - other.value)
        return NotImplemented
    
    def __repr__(self):
        return f"Point({self.value})"

class ContinuousInterval:
    def __init__(self, start, end, is_start_open=False, is_end_open=False):
        if start > end:
            raise ValueError("Invalid interval: start must be less or equal than end")

        if start == end and start != 0:
            empty_msg = f"Only start and end equal 0 is allowed!"
            error_msg = f"Invalid interval: open interval with zero length. {empty_msg}"
            raise ValueError(error_msg)

        self.start = start
        self.end = end
        self.is_start_open = is_start_open
        self.is_end_open = is_end_open
    
    @staticmethod
    def empty():
        return ContinuousInterval(0, 0, True, True)
    
    def is_empty(self):
        are_open=self.is_start_open and self.is_end_open
        are_zero=self.start == self.end and self.start == 0
        
        return are_zero and are_open

    def __eq__(self, other):
        if isinstance(other, ContinuousInterval):
            return (self.start, self.end, self.is_start_open, self.is_end_open) == \
                   (other.start, other.end, other.is_start_open, other.is_end_open)
        return NotImplemented

    def __ne__(self, other):
        if isinstance(other, ContinuousInterval):
            return (self.start, self.end, self.is_start_open, self.is_end_open) != \
                   (other.start, other.end, other.is_start_open, other.is_end_open)
        return NotImplemented

    def __lt__(self, other):
        if isinstance(other, ContinuousInterval):
            return self.end < other.start or (self.end == other.start and
                                               (self.is_end_open or other.is_start_open))
        return NotImplemented

    def __le__(self, other):
        if isinstance(other, ContinuousInterval):
            return self.end < other.end or (self.end == other.end and
                                             (self.is_end_open or not other.is_end_open))
        return NotImplemented

    def __gt__(self, other):
        if isinstance(other, ContinuousInterval):
            return other.__lt__(self)
        return NotImplemented

    def __ge__(self, other):
        if isinstance(other, ContinuousInterval):
            return other.__le__(self)
        return NotImplemented

    def __add__(self, other):
        if isinstance(other, ContinuousInterval):
            if self.is_empty():
                return other
            elif other.is_empty():
                return self
            elif self.end == other.start and not (self.is_end_open or other.is_start_open):
                return ContinuousInterval(self.start, other.end, self.is_start_open, other.is_end_open)
        return NotImplemented

    def __sub__(self, other):
        if isinstance(other, ContinuousInterval):
            if self.is_empty() or other.is_empty() or self == other:
                return ContinuousInterval.empty()
            elif other.end <= self.start or other.start >= self.end:
                return self
            elif self.start < other.start:
                if self.end > other.end:
                    return ContinuousInterval(self.start, other.start, self.is_start_open, not other.is_start_open) + \
                           ContinuousInterval(other.end, self.end, not other.is_end_open, self.is_end_open)
                else:
                    return ContinuousInterval(self.start, other.start, self.is_start_open, not other.is_start_open)
            else:
                return ContinuousInterval(other.end, self.end, not other.is_end_open, self.is_end_open)
        return NotImplemented
    
    def length(self):
        return self.end - self.start

    def contains(self, item):
        if isinstance(item, ContinuousInterval):
            return self.contains_interval(item)
        elif isinstance(item, Point):
            return self.contains_point(item)
        else:
            raise TypeError("Invalid type. Expected ContinuousInterval or Point.")

    def contains_interval(self, interval):
        if interval.start < self.start or interval.end > self.end:
            return False

        if interval.start == self.start and interval.is_start_open and not self.is_start_open:
            return False

        if interval.end == self.end and interval.is_end_open and not self.is_end_open:
            return False

        if interval.start == self.start and interval.end == self.end:
            return interval.is_start_open == self.is_start_open and interval.is_end_open == self.is_end_open

        return True

    def contains_point(self, point):
        return (point.value == self.start and not self.is_start_open) or \
               (point.value == self.end and not self.is_end_open) or \
               (self.start < point.value < self.end)

    def is_overlapping(self, interval):
        are_not_disjoint=not (self.end < interval.start or self.start > interval.end)
        endpoints_overlap=(self.end == interval.start and \
                           not self.is_end_open and not interval.is_start_open) or \
                         (self.start == interval.end and \
                          not self.is_start_open and not interval.is_end_open)
        has_intersection=(interval.start < self.start < interval.end) or \
                         (interval.start < self.end < interval.end) or \
                         (self.start < interval.start < self.end) or \
                         (self.start < interval.end < self.end)
        
        is_item_inside=self.contains(interval)
        
        return are_not_disjoint and (endpoints_overlap or has_intersection or is_item_inside)
    
    def intersection(self, interval):
        if not self.is_overlapping(interval) or interval.is_empty():
            return ContinuousInterval.empty()

        equal_endpoints = self.start == interval.start and self.end == interval.end
        equal_boundaries = self.is_start_open == interval.is_start_open and self.is_end_open == interval.is_end_open

        if equal_endpoints and equal_boundaries:
            # The intervals are coincidental
            return self

        if self.contains(interval):
            return interval

        if interval.contains(self):
            return self

        if self.start == interval.end and not self.is_start_open and not interval.is_end_open:
            return Point(self.start)

        if self.end == interval.start and not self.is_end_open and not interval.is_start_open:
            return Point(self.end)

        start = max(self.start, interval.start)
        end = min(self.end, interval.end)

        if start > end:
            return ContinuousInterval.empty()

        is_start_open = (start == self.start and self.is_start_open) or (start == interval.start and interval.is_start_open)
        is_end_open = (end == self.end and self.is_end_open) or (end == interval.end and interval.is_end_open)

        return ContinuousInterval(start, end, is_start_open, is_end_open)


    def union(self, interval):
        if not self.is_overlapping(interval):
            # Return the two disjoint intervals as a list
            return [self, interval]

        # Determine the start value
        if self.start < interval.start:
            start = self.start
            is_start_open = self.is_start_open
        elif self.start > interval.start:
            start = interval.start
            is_start_open = interval.is_start_open
        else:
            start = self.start
            is_start_open = self.is_start_open and interval.is_start_open

        # Determine the end value
        if self.end > interval.end:
            end = self.end
            is_end_open = self.is_end_open
        elif self.end < interval.end:
            end = interval.end
            is_end_open = interval.is_end_open
        else:
            end = self.end
            is_end_open = self.is_end_open and interval.is_end_open

        return ContinuousInterval(start, end, is_start_open, is_end_open)

    def difference(self, interval):
        if not self.is_overlapping(interval):
            return [self]

        if self.start >= interval.start and self.end <= interval.end:
            return []

        result = []

        if self.start < interval.start:
            start = self.start
            end = interval.start
            is_start_open = self.is_start_open
            is_end_open = interval.is_start_open
            result.append(ContinuousInterval(start, end, is_start_open, is_end_open))

        if self.end > interval.end:
            start = interval.end
            end = self.end
            is_start_open = interval.is_end_open
            is_end_open = self.is_end_open
            result.append(ContinuousInterval(start, end, is_start_open, is_end_open))

        return result
    
    def __repr__(self):
        input_msg=f"{self.start}, {self.end}, is_start_open={self.is_start_open}, is_end_open={self.is_end_open}"
        msg=f"ContinuousInterval({input_msg})"
        return msg

    
class DisjointInterval:
    def __init__(self, intervals):
        self.intervals = intervals

    def add_interval(self, interval):
        # Add a new continuous interval to the collection
        self.intervals.append(interval)

    def merge_overlapping_intervals(self):
        # Merge overlapping intervals within the collection
        merged_intervals = []
        sorted_intervals = sorted(self.intervals, key=lambda interval: interval.start)
        
        for interval in sorted_intervals:
            if not merged_intervals or merged_intervals[-1].end < interval.start:
                merged_intervals.append(interval)
            else:
                merged_intervals[-1].end = max(merged_intervals[-1].end, interval.end)
        
        self.intervals = merged_intervals

    def get_non_overlapping_intervals(self):
        # Retrieve a list of non-overlapping intervals
        self.merge_overlapping_intervals()
        return self.intervals

    def get_interval_containing_point(self, point):
        # Find the interval (if any) that contains the given point
        for interval in self.intervals:
            if interval.start <= point.value <= interval.end:
                return interval
        
        return None

class IntervalSet:
    def __init__(self, points, intervals, disjoint_intervals):
        self.points = points
        self.intervals = intervals
        self.disjoint_intervals = disjoint_intervals

    def find_intervals_containing_points(self, points):
        # Perform operations involving points and intervals together
        pass

    def merge_overlapping_intervals_within_disjoint_intervals(self):
        # Perform operations involving intervals and disjoint intervals together
        pass

In [20]:
%%ipytest --disable-warnings --color=yes --nb-coverage --log-cli-level=info

import pytest


@pytest.fixture
def point():
    return Point(5)

def test_empty_continuous_interval():
    empty_interval = ContinuousInterval.empty()

    assert empty_interval.start == 0
    assert empty_interval.end == 0
    assert empty_interval.is_start_open
    assert empty_interval.is_end_open
    assert empty_interval.is_empty()

@pytest.mark.parametrize("other, expected_result", [
    (Point(5), True),
    (Point(10), False),
])
def test_point_equality(point, other, expected_result):
    assert (point == other) == expected_result
    assert (point != other) == (not expected_result)


@pytest.mark.parametrize("other, expected_result", [
    (Point(10), True),
    (Point(5), False),
    (Point(2), False),
])
def test_point_comparison(point, other, expected_result):
    assert (point < other) == expected_result
    assert (point <= other) == expected_result or (point == other)
    assert (point > other) == (not expected_result) or (point == other)
    assert (point >= other) == (not expected_result) or (point == other)


@pytest.mark.parametrize("other, expected_result", [
    (Point(2), Point(7)),
])
def test_point_arithmetic_addition(point, other, expected_result):
    assert point + other == expected_result


@pytest.mark.parametrize("other, expected_result", [
    (Point(2), Point(3)),
])
def test_point_arithmetic_subtraction(point, other, expected_result):
    assert point - other == expected_result


@pytest.mark.parametrize("invalid_operation", [
    10,  # Comparison with incompatible type
    "test",  # Addition with incompatible type
    "test",  # Subtraction with incompatible type
])
def test_point_invalid_operations(point, invalid_operation):
    with pytest.raises(TypeError):
        point < invalid_operation

    with pytest.raises(TypeError):
        point + invalid_operation

    with pytest.raises(TypeError):
        point - invalid_operation

======================================= test session starts ========================================
platform linux -- Python 3.8.10, pytest-6.2.4, py-1.11.0, pluggy-0.13.1
rootdir: /home/brunolnetto/github/alloyha/experiments/data/continuousIntervals
plugins: anyio-3.6.2, pytest_notebook-0.8.1, cov-4.0.0, testmon-2.0.2
collected 11 items

t_c21d524c165d43d1be2589ccbd59b396.py::test_empty_continuous_interval PASSED                 [  9%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_point_equality[other0-True] PASSED               [ 18%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_point_equality[other1-False] PASSED              [ 27%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_point_comparison[other0-True] PASSED             [ 36%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_point_comparison[other1-False] PASSED            [ 45%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_point_comparison[other2-False] PASSED            [ 54%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_point_arithmet

In [16]:
%%ipytest --disable-warnings --color=yes --nb-coverage --log-cli-level=info

import pytest

@pytest.fixture
def interval(request):
    return request.param

@pytest.fixture
def interval1(request):
    return request.param

@pytest.fixture
def interval2(request):
    return request.param

# Test cases for length()
length_test_token = "interval, expected_length"
length_test_cases = [
    ((0, 5, False, False), 5),
    ((0, 10, True, False), 10),
    ((-5, 5, False, True), 10),
    ((-10, 10, True, True), 20),
    ((0, 0, False, False), 0),  # Zero-length interval
    ((0, 0, True, True), 0),  # Zero-length interval
]


@pytest.mark.parametrize(length_test_token, length_test_cases, indirect=["interval"])
def test_continuous_interval_length(interval, expected_length):
    start, end, is_start_open, is_end_open = interval
    interval_obj = ContinuousInterval(start, end, is_start_open, is_end_open)
    assert interval_obj.length() == expected_length

# Test cases for contains()
contains_test_token = "interval, item, expected_result"
contains_test_cases = [
    # Interval containment
    ((0, 5, False, False), (1, 4, False, False), True),
    ((0, 5, False, False), (0, 5, False, False), True),
    ((0, 5, False, False), (0, 5, True, True), False),
    ((0, 5, False, False), (3, 6, False, False), False),
    ((0, 5, False, True), (5, 10, False, False), False),
    ((0, 5, False, True), (0, 5, False, False), False),
    ((0, 5, False, True), (0, 5, True, True), False),
    ((0, 5, True, True), (0, 5, False, False), False),

    # Point containment
    ((0, 5, False, False), 2, True),
    ((0, 5, False, False), 0, True),
    ((0, 5, True, False), 0, False),
    ((0, 5, True, False), 5, True),
    ((0, 5, False, True), 5, False),
    ((0, 5, True, True), 0, False),
    ((-5, 5, False, False), -10, False),
    ((-5, 5, False, False), 10, False),
]

@pytest.mark.parametrize(contains_test_token, contains_test_cases)
def test_continuous_interval_contains(interval, item, expected_result):
    interval_obj = ContinuousInterval(*interval)

    if isinstance(item, tuple):
        # Interval containment
        interval_obj2 = ContinuousInterval(*item)
        assert interval_obj.contains(interval_obj2) == expected_result
    else:
        # Point containment
        point_obj = Point(item)
        assert interval_obj.contains(point_obj) == expected_result

is_overlapping_test_token = "interval1, interval2, expected_result"
# Test cases for is_overlapping()
is_overlapping_test_cases = [
    ((0, 5, False, False), (3, 8, False, False), True),
    ((0, 5, False, False), (5, 10, False, False), True),
    ((0, 5, False, False), (5, 10, True, False), False),
    ((0, 5, True, False), (5, 10, True, False), False),
    ((0, 5, False, False), (5, 10, False, True), True),
    ((0, 5, True, False), (5, 10, False, True), True),
    ((0, 5, True, False), (5, 10, True, True), False),
    ((0, 5, True, False), (10, 15, False, False), False),
]

@pytest.mark.parametrize(is_overlapping_test_token, is_overlapping_test_cases, indirect=["interval1", "interval2"])
def test_continuous_interval_is_overlapping(interval1, interval2, expected_result):
    start1, end1, is_start_open1, is_end_open1 = interval1
    start2, end2, is_start_open2, is_end_open2 = interval2
    
    interval_obj1 = ContinuousInterval(start1, end1, is_start_open1, is_end_open1)
    interval_obj2 = ContinuousInterval(start2, end2, is_start_open2, is_end_open2)
    
    print(interval_obj1.end < interval_obj2.start)
    print(interval_obj1.start > interval_obj2.end)
    print(interval_obj1.end == interval_obj2.start and not interval_obj1.is_end_open and not interval_obj2.is_start_open)
    print(interval_obj1.start == interval_obj2.end and not interval_obj1.is_start_open and not interval_obj2.is_end_open)
    
    assert interval_obj1.is_overlapping(interval_obj2) == expected_result



======================================= test session starts ========================================
platform linux -- Python 3.8.10, pytest-6.2.4, py-1.11.0, pluggy-0.13.1
rootdir: /home/brunolnetto/github/alloyha/experiments/data/continuousIntervals
plugins: anyio-3.6.2, pytest_notebook-0.8.1, cov-4.0.0, testmon-2.0.2
collected 30 items

t_c21d524c165d43d1be2589ccbd59b396.py::test_continuous_interval_length[interval0-5] PASSED   [  3%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_continuous_interval_length[interval1-10] PASSED  [  6%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_continuous_interval_length[interval2-10] PASSED  [ 10%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_continuous_interval_length[interval3-20] PASSED  [ 13%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_continuous_interval_length[interval4-0] PASSED   [ 16%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_continuous_interval_length[interval5-0] PASSED   [ 20%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_continuous_int

In [17]:
%%ipytest --disable-warnings --color=yes --nb-coverage --log-cli-level=info

intersection_test_token = "interval1, interval2, expected_result"
intersection_test_cases = [
    # No overlap
    ((0, 5, False, False), (6, 10, False, False), ContinuousInterval.empty()),
    ((0, 5, False, False), (5, 10, True, False), ContinuousInterval.empty()),
    ((0, 5, False, False), (0, 0, True, True), ContinuousInterval.empty()),  # Interval with zero length

    # Overlap resulting in a point
    ((0, 5, False, False), (5, 10, False, False), Point(5)),

    # Overlap resulting in a continuous interval
    ((0, 5, False, False), (2, 8, False, False), ContinuousInterval(2, 5, False, False)),
    ((0, 5, False, False), (2, 8, True, True), ContinuousInterval(2, 5, True, False)),
    ((0, 5, True, True), (2, 8, False, False), ContinuousInterval(2, 5, False, True)),
    ((0, 5, True, True), (2, 8, True, True), ContinuousInterval(2, 5, True, True)),

    # Overlap resulting in an empty interval
    ((0, 5, False, False), (5, 10, True, True), ContinuousInterval.empty()),
    ((0, 5, True, True), (5, 10, False, False), ContinuousInterval.empty()),
    ((0, 5, True, True), (5, 10, True, True), ContinuousInterval.empty()),
    
    # One or both intervals are empty
    ((0, 0, True, True), (1, 2, False, True), ContinuousInterval.empty()),
    ((1, 2, False, True), (0, 0, True, True), ContinuousInterval.empty()),
    ((0, 5, False, False), (0, 0, True, True), ContinuousInterval.empty()),
    ((0, 0, True, True), (0, 5, False, False), ContinuousInterval.empty()),
    
    # One interval is entirely contained within the other
    ((1, 9, False, False), (3, 6, False, False), ContinuousInterval(3, 6, False, False)),
    ((3, 6, False, False), (1, 9, False, False), ContinuousInterval(3, 6, False, False)),
    ((1, 9, False, False), (1, 9, False, False), ContinuousInterval(1, 9, False, False)),
    ((3, 6, False, False), (3, 6, False, False), ContinuousInterval(3, 6, False, False)),

    # Additional cases
    ((0, 5, False, False), (5, 10, True, True), ContinuousInterval.empty()),
    ((0, 5, True, True), (5, 10, False, False), ContinuousInterval.empty()),
    ((0, 5, True, True), (5, 10, True, True), ContinuousInterval.empty()),
    
    # One interval is entirely contained within the other
    ((1, 9, False, False), (1, 9, False, False), ContinuousInterval(1, 9, False, False)),
    ((3, 6, False, False), (3, 6, False, False), ContinuousInterval(3, 6, False, False)),
]

@pytest.mark.parametrize(intersection_test_token, intersection_test_cases, indirect=["interval1", "interval2"])
def test_continuous_interval_intersection(interval1, interval2, expected_result):
    interval_obj1 = ContinuousInterval(*interval1)
    interval_obj2 = ContinuousInterval(*interval2)
    expected_result_obj = (
        Point(*expected_result)
        if isinstance(expected_result, tuple)
        else expected_result
    )

    result = interval_obj1.intersection(interval_obj2)
    
    if isinstance(expected_result_obj, ContinuousInterval):
        assert isinstance(result, ContinuousInterval)
        assert result.start == expected_result_obj.start
        assert result.end == expected_result_obj.end
        assert result.is_start_open == expected_result_obj.is_start_open
        assert result.is_end_open == expected_result_obj.is_end_open
    elif isinstance(expected_result_obj, Point):
        assert isinstance(result, Point)
        assert result.value == expected_result_obj.value
    else:
        assert result is None

======================================= test session starts ========================================
platform linux -- Python 3.8.10, pytest-6.2.4, py-1.11.0, pluggy-0.13.1
rootdir: /home/brunolnetto/github/alloyha/experiments/data/continuousIntervals
plugins: anyio-3.6.2, pytest_notebook-0.8.1, cov-4.0.0, testmon-2.0.2
collected 24 items

t_c21d524c165d43d1be2589ccbd59b396.py::test_continuous_interval_intersection[interval10-interval20-expected_result0] PASSED [  4%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_continuous_interval_intersection[interval11-interval21-expected_result1] PASSED [  8%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_continuous_interval_intersection[interval12-interval22-expected_result2] PASSED [ 12%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_continuous_interval_intersection[interval13-interval23-expected_result3] PASSED [ 16%]
t_c21d524c165d43d1be2589ccbd59b396.py::test_continuous_interval_intersection[interval14-interval24-expected_result4] PASSED [ 20%]
t_c